#  Few-shot распознавание сущностей

Для обучения ner-моделей обычно нужно много данных. Но есть способ сократить требование к объёму данных - использовать предобученные языковые модели вроде Берта. Из-за того, что языковая модель даёт хорошие эмбединги, можно завести ner всего с 10 предложений.

Такой подхол называется Few-shot learning. Подробнее с тем, как это работает, мы разберемся, когда будем проходить нейроныые сети. А пока научимся пользоваться готовыми штуками, чтобы решать практические задачи. 

Берт и возможность тренировать few-shot ner модели есть в deep pavlov (http://docs.deeppavlov.ai/en/master/features/models/ner.html)

In [1]:
!pip install deeppavlov

     |████████████████████████████████| 735 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 5.6 MB/s  eta 0:00:01
     |████████████████████████████████| 1.2 MB 28.3 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 15.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 22.7 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 3.7 MB/s  eta 0:00:01
     |████████████████████████████████| 50 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 2.1 MB 26.6 MB/s eta 0:00:01
     |████████████████████████████████| 312 kB 30.2 MB/s eta 0:00:01
     |████████████████████████████████| 10.1 MB 31.9 MB/s eta 0:00:01
     |████████████████████████████████| 160 kB 37.4 MB/s eta 0:00:01
     |████████████████████████████████| 25.2 MB 23.3 MB/s eta 0:00:01     |██████████████▉                 | 11.7 MB 23.3 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 11.0 MB/s eta 0:00:01
     |███████████████████████████████

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=19852 sha256=8457f39b5f436f2dc40bfbdf41c10e6f23ec8b0ce1ad870a3a4ddbca3e241c14
  Stored in directory: /root/.cache/pip/wheels/3f/2a/fa/4d7a888e69774d5e6e855d190a8a51b357d77cc05eb1c097c9
  Created wheel for pyyaml: filename=PyYAML-5.3-cp36-cp36m-linux_x86_64.whl size=45519 sha256=3f624123c77ea715d569c1da06717c6bd67f1c70f6ff5d49814212e26aebd1de
  Stored in directory: /root/.cache/pip/wheels/b1/86/0d/10e6c39d3a2b85ba807d7657ee80f08cc16c03f2aa2adf8e46
  Created wheel for httptools: filename=httptools-0.0.13-cp36-cp36m-linux_x86_64.whl size=213776 sha256=88c68edbf4cfef05756a5b8645461c453fb8256dd8abdbe2d06fcc68f4dfdc76
  Stored in directory: /root/.cache/pip/wheels/10/f6/5e/1b89aee7bd3b1470f15c92054e684439ab17efb3914f63f104
  Created wheel for starlette: filename=starlette-0.12.8-py3-none-any.whl size=59349 sha256=776261b745f15a9c58ed73ea5ec6b167114147250a311081e9f9a1c587d006a2
  Stored in directory: /root/.cache/pip/

In [2]:
!pip install tensorflow==1.15.0

  Using cached tensorflow-1.15.0-cp36-cp36m-manylinux2010_x86_64.whl (412.3 MB)


In [3]:
!apt-get --yes install git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  git-man libcurl3-gnutls liberror-perl
Suggested packages:
  gettext-base git-daemon-run | git-daemon-sysvinit git-doc git-el git-email
  git-gui gitk gitweb git-cvs git-mediawiki git-svn
The following NEW packages will be installed:
  git git-man libcurl3-gnutls liberror-perl
0 upgraded, 4 newly installed, 0 to remove and 9 not upgraded.
Need to get 4950 kB of archives.
After this operation, 34.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libcurl3-gnutls amd64 7.58.0-2ubuntu3.8 [213 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 liberror-perl all 0.17025-1 [22.8 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 git-man all 1:2.17.1-1ubuntu0.5 [803 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 git amd64 1:2.17.1-1ubuntu0

In [8]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-zhp57c9m
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-zhp57c9m
  Running command git checkout -b feat/multi_gpu --track origin/feat/multi_gpu
  Switched to a new branch 'feat/multi_gpu'
  Branch 'feat/multi_gpu' set up to track remote branch 'feat/multi_gpu' from 'origin'.
  Created wheel for bert-dp: filename=bert_dp-1.0-cp36-none-any.whl size=20140 sha256=99ccc544941d70a8886a626f6ac5deb2aa51bc2add1739866b142d4b046b9bcb
  Stored in directory: /tmp/pip-ephem-wheel-cache-c1c9539p/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp


In [4]:
# загрузка файлов для колаба
# from google.colab import files
# uploaded = files.upload()

In [10]:
# в колабе файлы не перезаписываются, создается новый с (1,2,3..). 
# Чтобы удалить старый файл перед загрузкой нового используйте rm file
# rm .txt удаляет все txt файлы в текущей папке
#!rm *.txt

In [5]:
import pandas as pd

Возьмем тексты с авито по категории приставки. 

In [7]:
data = pd.read_csv('data/pristavki.csv', header=None, names=['text'])

Чтобы научиться выделять названия разметим небольшое количество текстов. Для DeepPavlov разметку нужно поделить на три файла: train.txt, valid.txt, test.txt.

In [9]:
!ls data/*.txt

data/test.txt  data/train.txt  data/valid.txt


Файлы для обучения, валидации и тестирования в формате IOB. B маркирует начало сущности, а I продолжение. O - обозначает несущность. PRS - это сокращение от PRISTAVKA. Можно было и полностью написать. 

In [10]:
!head data/train.txt

Продам O
PS3 B-PRS
, O
2 O
беспроводных O
джойстика O
в O
комплекте O

Продам O


Попробуем теперь обучить разметчик сущностей поверх многоязычного берта. Знать, что это такое и как работает не обязательно. Обязательно указать путь к обучающий файлам.

In [11]:
import json
from deeppavlov import configs, build_model, train_model

with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = 'data_games'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = 'data_games'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

ner_model = train_model(ner_config, download=True)

2020-01-21 20:07:07.856 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2020-01-21 20:07:07.867 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2020-01-21 20:07:10.861 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/data/tag.dict]


Using TensorFlow backend.






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use standard

2020-01-21 20:07:39.154 INFO in 'deeppavlov.models.bert.bert_sequence_tagger'['bert_sequence_tagger'] at line 251: [initializing model with Bert from /root/.deeppavlov/downloads/bert_models/multi_cased_L-12_H-768_A-12/bert_model.ckpt]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/downloads/bert_models/multi_cased_L-12_H-768_A-12/bert_model.ckpt


2020-01-21 20:07:45.371 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 50 tokens with 5 phrases; found: 8 phrases; correct: 0.

precision:  0.00%; recall:  0.00%; FB1:  0.00

	PRS: precision:  0.00%; recall:  0.00%; F1:  0.00 8


2020-01-21 20:07:45.372 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 198: Initial best ner_f1 of 0


{"valid": {"eval_examples_count": 5, "metrics": {"ner_f1": 0, "ner_token_f1": 33.9623}, "time_spent": "0:00:03", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 100}}


2020-01-21 20:08:09.460 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 338: Stopped training
2020-01-21 20:08:10.968 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/data/tag.dict]


KeyboardInterrupt: 

Обучается оно около 10 минут. Добавление примеров в обучение увеличит время обучения. 

Посмотрим как это все размечается.

In [11]:
ner_model(['Playstation 4', 'Xbox 360 продам', 'Продам ПС 3'])

[[['Playstation', '4'], ['Xbox', '360', 'продам'], ['Продам', 'ПС', '3']],
 [['B-PRS', 'I-PRS'], ['B-PRS', 'I-PRS', 'O'], ['O', 'B-PRS', 'I-PRS']]]

In [8]:
marked = []

for text in data.text.values[:1000]:
    # BERT имеет лимит на длину текста в 512 слов, возьмем даже еще меньше
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    # достанем только тексты с сущностями
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [9]:
marked

[[('Xbox', 'B-PRS'),
  ('в', 'O'),
  ('отличном', 'O'),
  ('состоянии', 'O'),
  (',', 'O'),
  ('2', 'O'),
  ('диска', 'O'),
  ('с', 'O'),
  ('играми', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('Поддержка', 'O'),
  ('носителей', 'O'),
  (':', 'O'),
  ('CD', 'O'),
  (',', 'O'),
  ('DVD', 'O'),
  (',', 'O'),
  ('флэшка', 'O'),
  ('.', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('Wi', 'B-PRS'),
  ('-', 'I-PRS'),
  ('Fi', 'I-PRS'),
  ('(', 'O'),
  ('802', 'O'),
  ('.', 'O'),
  ('11', 'O'),
  (')', 'O'),
  ('/', 'O'),
  ('\n', 'O'),
  ('Bluetooth', 'O'),
  ('.', 'O'),
  ('Цвет', 'O'),
  (':', 'O'),
  ('черный', 'O')],
 [('Sony', 'B-PRS'),
  ('PS3', 'B-PRS'),
  ('в', 'O'),
  ('отличном', 'O'),
  ('состоянии', 'O'),
  (',', 'O'),
  ('продаю', 'O'),
  ('за', 'O'),
  ('ненадобностью', 'O'),
  (',', 'O'),
  ('память', 'O'),
  ('12gb', 'O'),
  ('-', 'O'),
  ('встроенные', 'O'),
  ('+', 'O'),
  ('жесткий', 'O'),
  ('диск', 'O'),
  ('на', 'O'),
  ('160

Приставки конечно размечаются, но ещё и другие слова на английском начали захватываться. Можно попробовать добавить в данные примеры с ними, или сделать даже ещё пару типов сущностей, чтобы модель научилась их разделять.

В этой же моделе есть несколько стандартных сущностей.

In [13]:
ner_model = build_model(configs.ner.ner_ontonotes_bert_mult, download=True)

2019-12-19 12:25:41.729 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_ontonotes_bert_mult_v1.tar.gz to /root/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz
100%|██████████| 1.32G/1.32G [07:07<00:00, 3.09MB/s]
2019-12-19 12:32:48.739 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /root/.deeppavlov/ner_ontonotes_bert_mult_v1.tar.gz archive into /root/.deeppavlov/models
2019-12-19 12:33:02.850 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip download because of matching hashes
2019-12-19 12:33:03.371 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]
2019-12-19 12:33:28.634 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ontonotes

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model


In [14]:
ner_model(['Передай привет Михаилу Нефедову'])

[[['Передай', 'привет', 'Михаилу', 'Нефедову']],
 [['O', 'O', 'B-PERSON', 'I-PERSON']]]

In [16]:
ner_model(['Поезжай в Москву'])

[[['Поезжай', 'в', 'Москву']], [['O', 'O', 'B-GPE']]]

In [18]:
ner_model(['Поезжай в Альпы'])

[[['Поезжай', 'в', 'Альпы']], [['O', 'O', 'B-LOC']]]

In [19]:
ner_model(['Я говорю на русском и английском языках'])

[[['Я', 'говорю', 'на', 'русском', 'и', 'английском', 'языках']],
 [['O', 'O', 'O', 'B-LANGUAGE', 'O', 'B-LANGUAGE', 'O']]]

In [20]:
ner_model(['Бутылка воды стоит 50 рублей'])

[[['Бутылка', 'воды', 'стоит', '50', 'рублей']],
 [['O', 'O', 'O', 'B-MONEY', 'I-MONEY']]]

Но он конечно не идеальный.

In [40]:
ner_model(['ле Биг мак'])

[[['ле', 'Биг', 'мак']], [['B-PERSON', 'I-PERSON', 'I-PERSON']]]